# HuggingFace Transformers를 활용한 문장 분류 모델 학습

본 노트북에서는 `klue/roberta-base` 모델을 **KLUE** 내 **NLI** 데이터셋을 활용하여 모델을 훈련하는 예제를 다루게 됩니다.

학습을 통해 얻어질 `klue-roberta-base-nli` 모델은 입력된 두 문장의 추론 관계를 예측하는데 사용할 수 있게 됩니다.

학습 과정 이후에는 간단한 예제 코드를 통해 모델이 어떻게 활용되는지도 함께 알아보도록 할 것입니다.

모든 소스 코드는 [`huggingface-notebooks`](https://github.com/huggingface/notebooks)를 참고하였습니다.

먼저, 노트북을 실행하는데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 `transformers`가, 학습 데이터셋 로드를 위해서는 `datasets` 라이브러리의 설치가 필요합니다. 그 외 모델 성능 검증을 위해 `scipy`, `scikit-learn`을 추가로 설치해주도록 합니다.

In [ ]:
!pip install -U transformers datasets scipy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 7.9 MB/s eta 0:00:00


## 문장 분류 모델 학습

노트북을 실행하는데 필요한 라이브러리들을 모두 임포트합니다.

In [ ]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

학습에 필요한 정보를 변수로 기록합니다.

본 노트북에서는 `klue-roberta-base` 모델을 활용하지만, https://huggingface.co/klue 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

학습 태스크로는 `nli`를, 배치 사이즈로는 32를 지정하겠습니다.

In [ ]:
model_checkpoint = "klue/roberta-base"
batch_size = 32
task = "nli"

이제 HuggingFace `datasets` 라이브러리에 등록된 KLUE 데이터셋 중, NLI 데이터를 내려받습니다.

In [ ]:
datasets = load_dataset("klue", task)

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

다운로드 혹은 로드 후 얻어진 `datasets` 객체를 살펴보면, 훈련 데이터와 검증 데이터가 포함되어 있는 것을 확인할 수 있습니다.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 24998
    })
    validation: Dataset({
        features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
        num_rows: 3000
    })
})

각 예시 데이터는 아래와 같이 두 개의 문장과 두 문장의 추론 관계를 라벨로 지니고 있습니다.

In [ ]:
datasets["train"][0]

{'guid': 'klue-nli-v1_train_00000',
 'source': 'NSMC',
 'premise': '힛걸 진심 최고다 그 어떤 히어로보다 멋지다',
 'hypothesis': '힛걸 진심 최고로 멋지다.',
 'label': 0}

데이터셋을 전반적으로 살펴보기 위한 시각화 함수를 다음과 같이 정의합니다.

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.

**KLUE NLI**는 `entailment`, `neutral` 그리고 `contradiction` 세 개의 라벨을 지니는 데이터셋임을 확인할 수 있습니다.

In [ ]:
show_random_elements(datasets["train"])

,guid,source,premise,hypothesis,label
0,klue-nli-v1_train_04040,airbnb,근처에 피티 궁전과 괜찮은 식당들이 있습니다.,피티 궁전이 가까이에 있습니다.,entailment
1,klue-nli-v1_train_08273,wikinews,무죄 판결을 받은 서울대 교수에게 학교가 내린 해임처분은 정당하다는 법원 판결이 나왔다.,서울대 교수는 무죄 판결을 받는 것을 당연시했다.,neutral
2,klue-nli-v1_train_17032,wikitree,이에 고리본부는 주변 지역 내 구직자를 위한 양질의 일자리 창출을 최우선으로 삼고 지역과 상생하기 위한 정책을 추진하고 있다.,주변 지역은 고리본부의 상생 정책을 반기고 있다.,neutral
3,klue-nli-v1_train_08450,NSMC,미국 영화로서 최초로 7광구급 영화를 만들었다,최초로 7광구같은 영화를 제작했다.,entailment
4,klue-nli-v1_train_07480,airbnb,마리나는 친절하고 유쾌한 호스트입니다.,유쾌한 호스트는 마리나뿐입니다.,neutral
5,klue-nli-v1_train_23616,wikinews,한편 남경필 경기도지사의 아들이 행한 가혹행위는 6사단의 상급부대인 5군단에서 보강수사를 할 것이라고 밝혔습니다.,남경필 경기도지사의 아들이 행한 가혹행위에 대해 어떠한 수사도 없을 것이라고 밝혔습니다.,contradiction
6,klue-nli-v1_train_15507,NSMC,이 다큐는 방법과 마인드 자체가 삐뚤어졌다,다큐의 방법과 마인드가 올바르지 않다.,entailment
7,klue-nli-v1_train_08851,NSMC,반지의 제왕의 세계를 다시 만날 수 있다,반지의 제왕의 세계를 다시 볼 수 있다.,entailment
8,klue-nli-v1_train_08640,NSMC,밀양송전탑공사시위를 막았던 의경으로써 날조가 심하네요.,날조가 정말 심하네요.,entailment
9,klue-nli-v1_train_05479,wikipedia,"다음 날, 병원 의사들은 에드워드 박사가 병원에 오기 전 함께 일했던 여비서를 호출하고 그녀는 에드워드 박사의 사진을 의사들에게 보여준다.",에드워드 박사와 함께 일했던 여비서는 박사의 사진을 갖고 있었다.,entailment


훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

`datasets` 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 `load_metric` 함수가 있습니다.

그 중 **GLUE** 데이터셋에 이미 다양한 메트릭이 구현되어 있으므로, **GLUE** 그 중에서도 **KLUE NLI**와 동일한 `accuracy` 메트릭을 사용하는 `qnli` 태스크의 메트릭을 사용합니다.

In [ ]:
metric = load_metric("glue", "qnli")

<ipython-input-9-4b8a033d74b0>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "qnli")


`accuracy` 메트릭이 정상적으로 작동하는지 확인하기 위해, 랜덤한 예측 값과 라벨 값을 생성합니다.

In [ ]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
        0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0]),
 array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]))

앞서 생성한 랜덤 예측, 랜덤 라벨 값을 `compute()` 함수에 입력해 잘 동작하는지 확인해봅시다.

In [ ]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5}

이제 학습에 활용할 토크나이저를 로드해오도록 합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

로드된 토크나이저가 두 개 문장을 토큰화하는 방식을 파악하기 위해 두 문장을 입력 값으로 넣어줘보도록 합시다.

In [ ]:
tokenizer("힛걸 진심 최고로 멋지다.", "힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

{'input_ids': [0, 3, 7254, 3841, 2200, 11980, 2062, 18, 2, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

`input_ids`를 보시면 `cls_token`에 해당하는 2번 토큰이 가장 좌측에 붙게 되며, `sep_token`의 3번 토큰이 각각 중간과 가장 우측에 더해진 것을 확인할 수 있습니다.

이제 앞서 로드한 데이터셋에서 각 문장에 해당하는 *value* 를 뽑아주기 위한 *key* 를 정의합니다.

앞서 **KLUE NLI** 데이터셋의 두 문장은 각각 `premise`와 `hypothesis`라는 이름으로 정의된 것을 확인하였으니, 두 문장의 *key* 는 마찬가지로 각각 `premise`, `hypothesis`가 되게 됩니다.

In [ ]:
sentence1_key, sentence2_key = ("premise", "hypothesis")
print(f"Sentence 1: {datasets['train'][0][sentence1_key]}")
print(f"Sentence 2: {datasets['train'][0][sentence2_key]}")

Sentence 1: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다
Sentence 2: 힛걸 진심 최고로 멋지다.


이제 *key* 도 확인이 되었으니, 데이터셋에서 각 예제들을 뽑아와 토큰화 할 수 있는 함수를 아래와 같이 정의해줍니다.

해당 함수는 모델을 훈련하기 앞서 데이터셋을 미리 토큰화 시켜놓는 작업을 위한 콜백 함수로 사용되게 됩니다.

인자로 넣어주는 `truncation`는 모델이 입력 받을 수 있는 최대 길이 이상의 토큰 시퀀스가 들어오게 될 경우, 최대 길이 기준으로 시퀀스를 자르라는 의미를 지닙니다.

( \* `return_token_type_ids`는 토크나이저가 `token_type_ids`를 반환하도록 할 것인지를 결정하는 인자입니다. `transformers==4.7.0` 기준으로 `token_type_ids`가 기본적으로 반환되므로 `token_type_ids` 자체를 사용하지 않는 `RoBERTa` 모델을 활용하기 위해 해당 인자를 `False`로 설정해주도록 합니다.)

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        truncation=True,
        return_token_type_ids=False,
    )

앞서 정의한 `process_function`은 여러 개의 예제 데이터를 받을 수도 있습니다.

In [ ]:
preprocess_function(datasets["train"][:5])

{'input_ids': [[0, 3, 7254, 3841, 2062, 636, 3711, 12717, 2178, 2062, 11980, 2062, 2, 3, 7254, 3841, 2200, 11980, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 1525, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 1282, 2955, 3308, 2052, 3944, 11580, 2359, 2062, 18, 2], [0, 3911, 2377, 2366, 1521, 3061, 4785, 1282, 2955, 3308, 3515, 2170, 22, 2532, 5675, 2, 3911, 2377, 2366, 5105, 2318, 831, 717, 2886, 2069, 575, 555, 2062, 18, 2], [0, 10522, 2548, 2500, 6328, 2170, 6189, 5916, 4015, 2116, 1039, 2219, 3606, 18, 2, 10522, 2548, 2500, 6328, 27135, 5916, 4015, 1642, 2015, 2259, 4258, 2219, 3606, 18, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1

이제 정의된 전처리 함수를 활용해 데이터셋을 미리 토큰화시키는 작업을 수행합니다.

`datasets` 라이브러리를 통해 얻어진 `DatasetDict` 객체는 `map()` 함수를 지원하므로, 정의된 전처리 함수를 데이터셋 토큰화를 위한 콜백 함수로 `map()` 함수 인자로 넘겨주면 됩니다.

보다 자세한 내용은 [문서](https://huggingface.co/docs/datasets/processing.html#processing-data-with-map)를 참조해주시면 됩니다.

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

학습을 위한 모델을 로드할 차례입니다.

앞서 살펴본 바와 같이 **KLUE NLI**에는 총 3개의 클래스가 존재하므로, 3개의 클래스를 예측할 수 있는 *SequenceClassification* 구조로 모델을 로드하도록 합니다.

In [ ]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classif

모델을 로드할 때 발생하는 경고 문구는 두 가지 의미를 지닙니다.

1. *Masked Language Modeling* 을 위해 존재했던 `lm_head`가 현재는 사용되지 않고 있음을 의미합니다.
2. 문장 분류를 위한 `classifier` 레이어를 백본 모델 뒤에 이어 붙였으나 아직 훈련이 되지 않았으므로, 학습을 수행해야 함을 의미합니다.

마지막으로 앞서 정의한 메트릭을 모델 예측 결과에 적용하기 위한 함수를 정의합니다.

입력으로 들어오는 `eval_pred`는 [*EvalPrediction*](https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction) 객체이며, 모델의 클래스 별 예측 값과 정답 값을 지닙니다.

클래스 별 예측 중 가장 높은 라벨을 `argmax()`를 통해 뽑아낸 후, 정답 라벨과 비교를 하게 됩니다.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

이제 앞서 정의한 정보들을 바탕으로 `transformers`에서 제공하는 *Trainer* 객체를 활용하기 위한 인자 관리 클래스를 초기화합니다.

`metric_name`은 앞서 얻어진 메트릭 함수를 활용했을 때, 아래와 같이 `dict` 형식으로 결과 값이 반환되는데 여기서 우리가 사용할 *key* 를 정의해준다고 생각하시면 됩니다.

```python
>>> metric.compute(predictions=fake_preds, references=fake_labels)
{'accuracy': 0.515625}
```

각 인자에 대한 자세한 설명은 [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)에서 참조해주시면 됩니다.

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-nli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

이제 로드한 모델, 인자 관리 클래스, 데이터셋 등을 *Trainer* 클래스를 초기화에 넘겨주도록 합니다.

(TIP: Q: 이미 `encoded_datasets`을 만드는 과정에 토큰화가 이루어졌는데 토크나이저를 굳이 넘겨주는 이유가 무엇인가요?,<br>A: 토큰화는 이루어졌지만 학습 과정 시, 데이터를 배치 단위로 넘겨주는 과정에서 배치에 포함된 가장 긴 시퀀스 기준으로 `truncation`을 수행하고 최대 길이 시퀀스 보다 짧은 시퀀스들은 그 길이만큼 `padding`을 수행해주기 위함입니다.)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

이제 정의된 *Trainer* 객체를 다음과 같이 훈련시킬 수 있습니다.

에폭이 지남에 따라 *Loss* 는 떨어지고, 앞서 선정한 메트릭인 *Accuracy* 는 증가하는 것을 확인할 수 있습니다.

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

*Trainer* 는 학습을 마치게 되면, `load_best_model_at_end=True` 인자에 따라 메트릭 기준 가장 좋은 성능을 보였던 체크포인트를 로드하게 됩니다.

본 노트북에서는 마지막 에폭 때 가장 좋은 성능을 얻었기에 `evaluate`를 수행해도 같은 결과가 나오겠습니다.

In [ ]:
trainer.evaluate()

{'epoch': 5.0,
 'eval_accuracy': 0.8663333333333333,
 'eval_loss': 0.6880185604095459,
 'eval_runtime': 12.7427,
 'eval_samples_per_second': 235.429,
 'eval_steps_per_second': 7.377}

지금까지 `transformers`를 라이브러리 내 문장 분류 모델을 학습하는 과정을 KLUE NLI 데이터셋을 통해 알아보았습니다.

본 노트북을 통해 습득한 지식이 여러분의 업무와 학습에 도움이 되었으면 좋겠습니다.

```
허 훈 (huffonism@gmail.com)
```

APPENDIX: 앞서 학습된 모델을 HuggingFace 모델 허브에 업로드하였으니, 아래 예제와 같이 `pipeline` 함수를 통해 사용이 가능합니다.

먼저 `text-classification` 태스크로 파이프라인 객체를 초기화합니다.

( \* `return_all_scores`는 모델이 입력 문장에 대해 측정한 각 라벨에 대한 확률 값을 모두 보여줄 것인지를 결정하는 인자입니다.)

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="Huffon/klue-roberta-base-nli",
    return_all_scores=True,
)

NLI는 두 문장의 페어를 입력 값으로 주어야 하기 때문에 구분자 스페셜 토큰을 두 문장 사이에 넣어주기 위해 토크나이저 객체를 로드합니다.

`[SEP]` 문자열을 하드코딩하여 넣어줄 수도 있겠지만, 스페셜 토큰은 토크나이저 마다 다르게 정의되므로 다른 모델을 활용할 때 보다 코드를 재사용할 수 있도록 토크나이저의 `sep_token` 프로퍼티에 접근하는 방식으로 코드를 작성합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")

In [ ]:
tokenizer.sep_token

'[SEP]'

`sep_token`으로 두 문장을 이어 하나의 입력 값으로 파이프라인에 넘겨줍니다.

입력된 문장에 대해 모델이 각 라벨에 대해 어떤 확률을 가지고 예측했는지를 확인할 수 있습니다.

In [ ]:
classifier(f"흡연하려면 발코니 있는 방을 선택하면 됩니다. {tokenizer.sep_token} 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.")

[[{'label': 'ENTAILMENT', 'score': 0.9865273833274841},
  {'label': 'NEUTRAL', 'score': 0.013215206563472748},
  {'label': 'CONTRADICTION', 'score': 0.0002573762903921306}]]

In [ ]:
classifier(f"호스트분은 영어밖에 못하십니다. {tokenizer.sep_token} 호스트분도 엄청 친절하시고 영어도 잘하십니다.")

[[{'label': 'ENTAILMENT', 'score': 0.0002685467479750514},
  {'label': 'NEUTRAL', 'score': 0.0006742384284734726},
  {'label': 'CONTRADICTION', 'score': 0.9990572333335876}]]

검증 데이터에 존재하는 임의의 문장 페어를 입력해보니, 우리가 원하던 결과를 얻을 수 있었습니다.

*(cf. NLI 데이터에 대해 학습된 모델을 활용해 Zero-shot Classification을 수행하는 예제가 궁금하신 분들은 해당 [노트북](https://colab.research.google.com/github/Huffon/klue-transformers-tutorial/blob/master/zero_shot_classification.ipynb)을 참조해주세요.)*

In [ ]:
#출처 : https://github.com/Huffon/klue-transformers-tutorial